In [2]:
#I want to get to know gradient boosting methods (in particular, the xgboost library) and i am also currently in barbados.
#Import libraries:
import numpy as np
import pandas as pd
import xgboost as xgb
import time

# Stratified CV w/ XGBoost

Loading & preprocessing:

In [3]:
#https://www.kaggle.com/director/bnp-paribas-cardif-claims-management/simple-xgboost-0-46146/code
print('Load data...')
train = pd.read_csv("train.csv")
target = train['target']
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
ids = test['ID'].values
test = test.drop(['ID'],axis=1)
#
print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = train_series.mean()
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = train_series.mean()  #TODO

Load data...
Clearing...


In [ ]:
#https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/forums/t/19083/best-practices-for-parameter-tuning-on-models/
#https://github.com/dmlc/xgboost/blob/master/demo/guide-python/cross_validation.py

In [48]:
t0 = time.time()
xgtrain = xgb.DMatrix(train.values, target.values)
xgtest = xgb.DMatrix(test.values)

params = {'objective': 'binary:logistic', 
              'subsample': 1, 
              'eta': 0.1, 
              'colsample_bytree': 0.9, 
              'max_depth': 10,
              'min_child_weight' : 5,
                 'silent':1}

#Now let's fit the model
print('Fit the model...')
num_round = 50 #1800 CHANGE THIS BEFORE START
clf = xgb.cv(params,xgtrain,num_boost_round=num_round,metrics={'logloss'}, nfold = 5 ,
             seed = 0 ,maximize=False)

#i have attempted this with argument stratified = 1 and get the following error:
#TypeError: cv() got an unexpected keyword argument 'stratified'


#Make predict
# print('Predict...')
##check here for eval metrics + https://github.com/dmlc/xgboost/blob/master/demo/guide-python/evals_result.py
t1 = time.time()
total_time = t1 - t0
print total_time

Fit the model...
408.922273874


In [49]:
clf

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.657933,0.000591,0.655429,0.000550
1,0.628350,0.000739,0.623086,0.000943
2,0.603884,0.001104,0.595802,0.001063
3,0.583370,0.001012,0.572541,0.000836
4,0.566755,0.000924,0.553194,0.000422
5,0.552614,0.000855,0.536283,0.000151
6,0.540211,0.000855,0.521392,0.000222
7,0.529721,0.000880,0.508403,0.000270
8,0.520790,0.000867,0.497102,0.000514
9,0.513212,0.000905,0.487175,0.000640


# Some notes on xgb.train() 

In [82]:
from sklearn.cross_validation import KFold, train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)

In [84]:
xgtrains = xgb.DMatrix(X_train.values, y_train.values)
xgtest = xgb.DMatrix(X_test.values, y_test.values)
# xgtest = xgb.DMatrix(test.values)
params = {'objective': 'binary:logistic', 
              'subsample': 1, 
              'eta': 0.1, 
              'colsample_bytree': 0.9, 
              'max_depth': 10,
              'min_child_weight' : 5,
                 'silent':1}

In [85]:
clft = xgb.train(params,xgtrains,num_boost_round=num_round,
                 evals= [(xgtest,'logloss')] , early_stopping_rounds = 10,
                 verbose_eval=True)

Will train until logloss error hasn't decreased in 10 rounds.
[0]	logloss-error:0.235576
[1]	logloss-error:0.224310
[2]	logloss-error:0.224205
[3]	logloss-error:0.223015
[4]	logloss-error:0.223260
[5]	logloss-error:0.222980
[6]	logloss-error:0.222246
[7]	logloss-error:0.221721
[8]	logloss-error:0.220601
[9]	logloss-error:0.220076
[10]	logloss-error:0.220531
[11]	logloss-error:0.221056
[12]	logloss-error:0.220426
[13]	logloss-error:0.219761
[14]	logloss-error:0.219901
[15]	logloss-error:0.219621
[16]	logloss-error:0.219481
[17]	logloss-error:0.219062
[18]	logloss-error:0.219307
[19]	logloss-error:0.218922
[20]	logloss-error:0.219307
[21]	logloss-error:0.219307
[22]	logloss-error:0.219132
[23]	logloss-error:0.219272
[24]	logloss-error:0.219237
[25]	logloss-error:0.218887
[26]	logloss-error:0.218992
[27]	logloss-error:0.219167
[28]	logloss-error:0.218642
[29]	logloss-error:0.217802
[30]	logloss-error:0.217732
[31]	logloss-error:0.217627
[32]	logloss-error:0.217592
[33]	logloss-error:0.218